## Preparation of Flux-Free Kitaev Honeycomb lattice state

2 honeycombs

N = 14 spins

Hamiltonian: $H = -J_x \sum_{\langle i,j \rangle_x} \sigma_i^x \sigma_j^x
    -J_y \sum_{\langle i,j \rangle_y} \sigma_i^y \sigma_j^y
    -J_z \sum_{\langle i,j \rangle_z} \sigma_i^z \sigma_j^z$ 

Step 1: prepare product state $|00..>$ of n = N/2 spins

In [2]:
import numpy as np
import scipy

We are doing this mapping:

In [19]:
n_spins = 7
e1 = np.array([1,0])
psi = e1
for _ in range(n_spins-1):
    psi = np.kron(psi,e1)

assert len(psi) == 2**7

In [ ]:
print(2**7) 

128


Identify a representative qubit in each plaquette!